In [0]:
# GUIDED CAPSTONE STEP 4: THIS NOTEBOOK TAKES THE PARQUET FILES IN BLOB STORAGE AND PROVIDES SOME METRICS (ANALYTICAL ETL)
# GOAL: TAKE THE QUOTES FOR TODAY, APPEND PREVIOUS DAY'S LAST TRADE PRICE AND LAST TRADE PRICE FROM TODAY

In [0]:
# packages used
import math
import os
from pyspark.sql.functions import col, row_number, desc, concat, rank
from pyspark.sql.window import Window

In [0]:
# Azure credentials
storageAccountName = 'saderekguidedcapstone'
storageAccountAccessKey = <storageAccountAccessKey>
blobContainerName = 'container1'

spark.conf.set(
    f'fs.azure.account.key.{storageAccountName}.blob.core.windows.net',
    storageAccountAccessKey
)

In [0]:
# SKIP
def calculate_trade_moving_averages(exchange, date):
  # exchange = 'nyse', 'nasdaq'
  # date = '20200805', '20200806'

  # read parquet file into dataframe
  df = spark.read.parquet('wasbs://{}@{}.blob.core.windows.net/data/output/{}/trade/trade_dt={}'.format(blobContainerName, storageAccountName, exchange, date))
  
  # write temporary Spark view
  df.select(col('symbol'), col('exchange'), col('event_tm'), col('event_seq_nb'), col('trade_pr')).createOrReplaceTempView('tmp_trade_moving_avg')
  
  # calculate 30-min moving averages
  mov_avg_df = spark.sql('SELECT *, AVG(trade_pr) OVER(PARTITION BY symbol ORDER BY event_tm RANGE BETWEEN INTERVAL 30 MINUTES PRECEDING AND CURRENT ROW) AS mov_avg_trade_pr FROM tmp_trade_moving_avg')
  
  # write Hive staging table
  mov_avg_df.write.saveAsTable('temp_trade_moving_avg')
  
  return

In [0]:
# testing
spark.sql('SHOW TABLES').show()

+--------+--------------------+-----------+
database| tableName|isTemporary|
+--------+--------------------+-----------+
 default|temp_prev_day_las...| false|
 default|temp_previous_day...| false|
 default|temp_quote_moving...| false|
 default| temp_todays_trades| false|
 default|temp_trade_moving...| false|
 |tmp_previous_day_...| true|
+--------+--------------------+-----------+

In [0]:
# 1/3
def create_table_todays_quotes(exchange, date):
  # exchange = 'nyse', 'nasdaq'
  # date = '20200805', '20200806'

  # read parquet file into dataframe
  df = spark.read.parquet('wasbs://{}@{}.blob.core.windows.net/data/output/{}/quote/quote_dt={}'.format(blobContainerName, storageAccountName, exchange, date))

  # append event_seq_nb from related trade
  df2 = df.withColumn('event_seq_nb_tr_temp', df.event_seq_nb/10)
  df3 = df2.withColumn('event_seq_nb_tr', df2.event_seq_nb_tr_temp.cast('int')*10).drop('event_seq_nb_tr_temp')
  
  # write Hive staging table
  df3.write.saveAsTable('temp_todays_quotes')
  
  return

create_table_todays_quotes('nyse', '20200806')
spark.sql('SELECT * FROM temp_todays_quotes').show()

+----------+------+--------+--------------------+------------+-------------------+---------+--------+---------+--------+---------------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb| arrival_tm| bid_pr|bid_size| ask_pr|ask_size|event_seq_nb_tr|
+----------+------+--------+--------------------+------------+-------------------+---------+--------+---------+--------+---------------+
2020-08-06| SYMA| NYSE|2020-08-06 09:39:...| 1|2020-08-06 09:30:00| 77.67913| 100|78.437355| 100| 0|
2020-08-06| SYMA| NYSE|2020-08-06 09:47:...| 2|2020-08-06 09:30:00| 76.53373| 100| 76.94425| 100| 0|
2020-08-06| SYMA| NYSE|2020-08-06 09:56:...| 3|2020-08-06 09:30:00|75.120605| 100| 75.39408| 100| 0|
2020-08-06| SYMA| NYSE|2020-08-06 10:03:...| 4|2020-08-06 09:30:00| 74.86369| 100| 75.76861| 100| 0|
2020-08-06| SYMA| NYSE|2020-08-06 10:09:...| 5|2020-08-06 09:30:00| 77.7765| 100|78.801094| 100| 0|
2020-08-06| SYMA| NYSE|2020-08-06 10:18:...| 6|2020-08-06 09:30:00| 75.02947| 100|75.690575| 100| 0|
2020-08-06| SYMA| NYSE|2020-08-06 10:25:...| 7|2020-08-06 09:30:00| 75.36016| 100| 75.46482| 100| 0|
2020-08-06| SYMA| NYSE|2020-08-06 10:34:...| 8|2020-08-06 09:30:00| 78.58505| 100| 78.60169| 100| 0|
2020-08-06| SYMA| NYSE|2020-08-06 10:41:...| 9|2020-08-06 09:30:00| 75.7057| 100| 76.89216| 100| 0|
2020-08-06| SYMA| NYSE|2020-08-06 10:55:...| 11|2020-08-06 09:30:00|78.310555| 100| 79.47636| 100| 10|
2020-08-06| SYMA| NYSE|2020-08-06 11:03:...| 12|2020-08-06 09:30:00| 77.97817| 100| 79.28954| 100| 10|
2020-08-06| SYMA| NYSE|2020-08-06 11:12:...| 13|2020-08-06 09:30:00|75.007744| 100| 75.3299| 100| 10|
2020-08-06| SYMA| NYSE|2020-08-06 11:20:...| 14|2020-08-06 09:30:00| 75.20346| 100| 77.09724| 100| 10|
2020-08-06| SYMA| NYSE|2020-08-06 11:27:...| 15|2020-08-06 09:30:00| 75.49015| 100| 76.72422| 100| 10|
2020-08-06| SYMA| NYSE|2020-08-06 11:32:...| 16|2020-08-06 09:30:00|77.776596| 100| 78.86235| 100| 10|
2020-08-06| SYMA| NYSE|2020-08-06 11:39:...| 17|2020-08-06 09:30:00| 77.3139| 100| 79.10032| 100| 10|
2020-08-06| SYMA| NYSE|2020-08-06 11:45:...| 18|2020-08-06 09:30:00|79.173676| 100| 80.35081| 100| 10|
2020-08-06| SYMA| NYSE|2020-08-06 11:53:...| 19|2020-08-06 09:30:00|77.161835| 100| 78.59093| 100| 10|
2020-08-06| SYMA| NYSE|2020-08-06 12:09:...| 21|2020-08-06 09:30:00| 74.88462| 100| 75.63901| 100| 20|
2020-08-06| SYMA| NYSE|2020-08-06 12:18:...| 22|2020-08-06 09:30:00| 76.77406| 100|78.560974| 100| 20|
+----------+------+--------+--------------------+------------+-------------------+---------+--------+---------+--------+---------------+
only showing top 20 rows

In [0]:
# 2/3
def create_table_todays_trades(exchange, date):
  # exchange = 'nyse', 'nasdaq'
  # date = '20200805', '20200806'

  # read parquet file into dataframe
  df = spark.read.parquet('wasbs://{}@{}.blob.core.windows.net/data/output/{}/trade/trade_dt={}'.format(blobContainerName, storageAccountName, exchange, date))

  # write Hive staging table
  df.write.saveAsTable('temp_todays_trades')
  
  return

create_table_todays_trades('nyse', '20200806')
spark.sql('SELECT * FROM temp_todays_trades').show()

+----------+------+--------+--------------------+------------+-------------------+---------+----------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb| arrival_tm| trade_pr|trade_size|
+----------+------+--------+--------------------+------------+-------------------+---------+----------+
2020-08-06| SYMA| NYSE|2020-08-06 10:49:...| 10|2020-08-06 09:30:00|74.493774| 27|
2020-08-06| SYMA| NYSE|2020-08-06 12:00:...| 20|2020-08-06 09:30:00|76.161194| 315|
2020-08-06| SYMA| NYSE|2020-08-06 13:11:...| 30|2020-08-06 09:30:00|76.899635| 755|
2020-08-06| SYMA| NYSE|2020-08-06 14:27:...| 40|2020-08-06 09:30:00| 77.11552| 81|
2020-08-06| SYMA| NYSE|2020-08-06 15:39:...| 50|2020-08-06 09:30:00| 76.37442| 744|
2020-08-06| SYMA| NYSE|2020-08-06 16:58:...| 60|2020-08-06 09:30:00| 78.32364| 857|
2020-08-06| SYMA| NYSE|2020-08-06 18:14:...| 70|2020-08-06 09:30:00| 78.23471| 125|
2020-08-06| SYMA| NYSE|2020-08-06 19:28:...| 80|2020-08-06 09:30:00|76.528984| 98|
2020-08-06| SYMA| NYSE|2020-08-06 20:49:...| 90|2020-08-06 09:30:00| 75.71484| 546|
2020-08-06| SYMA| NYSE|2020-08-06 22:00:...| 100|2020-08-06 09:30:00| 76.30847| 471|
2020-08-06| SYMB| NYSE|2020-08-06 10:45:...| 10|2020-08-06 09:30:00| 33.85934| 314|
2020-08-06| SYMB| NYSE|2020-08-06 12:01:...| 20|2020-08-06 09:30:00| 32.92886| 270|
2020-08-06| SYMB| NYSE|2020-08-06 13:13:...| 30|2020-08-06 09:30:00|33.686092| 792|
2020-08-06| SYMB| NYSE|2020-08-06 14:29:...| 40|2020-08-06 09:30:00| 35.07168| 852|
2020-08-06| SYMB| NYSE|2020-08-06 15:46:...| 50|2020-08-06 09:30:00|34.834843| 441|
2020-08-06| SYMB| NYSE|2020-08-06 16:57:...| 60|2020-08-06 09:30:00| 33.31802| 773|
2020-08-06| SYMB| NYSE|2020-08-06 18:06:...| 70|2020-08-06 09:30:00|33.565834| 578|
2020-08-06| SYMB| NYSE|2020-08-06 19:21:...| 80|2020-08-06 09:30:00| 33.10892| 718|
2020-08-06| SYMB| NYSE|2020-08-06 20:36:...| 90|2020-08-06 09:30:00|32.643288| 220|
2020-08-06| SYMB| NYSE|2020-08-06 21:46:...| 100|2020-08-06 09:30:00|35.924744| 741|
+----------+------+--------+--------------------+------------+-------------------+---------+----------+
only showing top 20 rows

In [0]:
# 3/3
def create_table_prev_day_last_trades(exchange, date):
  # pull previous day's trades
  df = spark.read.parquet('wasbs://{}@{}.blob.core.windows.net/data/output/{}/trade/trade_dt={}'.format(blobContainerName, storageAccountName, exchange, str(int(date) - 1)))
  
  # write them to a view
  df.createOrReplaceTempView('tmp_previous_day_trades')
  
  # filter for the last trade within each symbol
  prev_day_last_trades = spark.sql('SELECT trade_dt, symbol, exchange, event_tm, event_seq_nb, arrival_tm, trade_pr, trade_size FROM (SELECT *, ROW_NUMBER() OVER(PARTITION BY symbol ORDER BY event_seq_nb DESC) AS rn FROM tmp_previous_day_trades) WHERE rn=1')
  
  # write the last trades to a table
  prev_day_last_trades.write.saveAsTable('temp_prev_day_last_trades')
  
  return

create_table_prev_day_last_trades('nyse', '20200806')
spark.sql('SELECT * FROM temp_prev_day_last_trades').show()

+----------+------+--------+--------------------+------------+-------------------+---------+----------+
 trade_dt|symbol|exchange| event_tm|event_seq_nb| arrival_tm| trade_pr|trade_size|
+----------+------+--------+--------------------+------------+-------------------+---------+----------+
2020-08-05| SYMA| NYSE|2020-08-05 21:30:...| 100|2020-08-05 09:30:00| 77.78611| 189|
2020-08-05| SYMB| NYSE|2020-08-05 21:27:...| 100|2020-08-05 09:30:00|33.956287| 369|
2020-08-05| SYMC| NYSE|2020-08-05 21:52:...| 100|2020-08-05 09:30:00|160.61949| 312|
+----------+------+--------+--------------------+------------+-------------------+---------+----------+

In [0]:
def write_analytical(exchange, date):
  df = spark.sql('''
  SELECT q.trade_dt, q.exchange, q.symbol, q.event_tm, q.event_seq_nb, q.bid_pr, q.bid_size, q.ask_pr, q.ask_size, tp.trade_pr AS prev_day_last_trade, t.trade_pr AS today_last_trade
  FROM temp_todays_quotes AS q
  INNER JOIN temp_prev_day_last_trades AS tp USING(symbol)
  LEFT JOIN temp_todays_trades AS t ON q.event_seq_nb_tr=t.event_seq_nb AND q.symbol=t.symbol
  ''')

  df.write.parquet('wasbs://{}@{}.blob.core.windows.net/analytical/{}/date={}'.format(blobContainerName, storageAccountName, exchange, date))

  return

write_analytical('nyse', '20200806')
write_analytical('nasdaq', '20200806')